In [1]:
from langchain_groq import ChatGroq

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv('GROQ_API_KEY')

In [3]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key=GROQ_API_KEY, 
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [27]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/director-software-engineering-itc/job/R-49416")
page_data = loader.load().pop().page_content

In [28]:
print(page_data)






















Director Software Engineering, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Sel

In [29]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Director Software Engineering, ITC",
  "experience": "14+ years experience leading high performing software engineering teams",
  "skills": [
    "Cloud native technologies",
    "Cloud providers like AWS, Azure",
    "Agile software development methodology",
    "DevOps (CICD)",
    "Modern software architectural principles",
    "Front-end web application tech (e.g. javascript, CSS, html5, Vue, React/redux)",
    "Distributed cloud systems tech (e.g. node.js, EC2, Kubernetes, Lambda, Bedrock, DynamoDB, Elasticsearch)",
    "AI, Machine Learning and related data solutions",
    "Microservices and applications"
  ],
  "description": "The Product Innovation and Consumer Creation team provides the foundation for how we make Nike product; from innovation, to design, development, line planning, and merchandising. We are looking for a Director of Software Engineering to lead teams that build and develop web experiences, core platform APIs leveraging AI/ML for Nike’s Mer

In [30]:
type(res.content)

str

In [31]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

print(json_res)

{'role': 'Director Software Engineering, ITC', 'experience': '14+ years experience leading high performing software engineering teams', 'skills': ['Cloud native technologies', 'Cloud providers like AWS, Azure', 'Agile software development methodology', 'DevOps (CICD)', 'Modern software architectural principles', 'Front-end web application tech (e.g. javascript, CSS, html5, Vue, React/redux)', 'Distributed cloud systems tech (e.g. node.js, EC2, Kubernetes, Lambda, Bedrock, DynamoDB, Elasticsearch)', 'AI, Machine Learning and related data solutions', 'Microservices and applications'], 'description': 'The Product Innovation and Consumer Creation team provides the foundation for how we make Nike product; from innovation, to design, development, line planning, and merchandising. We are looking for a Director of Software Engineering to lead teams that build and develop web experiences, core platform APIs leveraging AI/ML for Nike’s Merchandising functions.'}


In [32]:
type(res.content)

str

In [33]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [34]:
import uuid
import chromadb

# if use PersistentClient method then it will be creating a folder (which you pass as a parameter) otherwise it will create in memory
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [35]:
job=json_res

In [36]:
job['skills']

['Cloud native technologies',
 'Cloud providers like AWS, Azure',
 'Agile software development methodology',
 'DevOps (CICD)',
 'Modern software architectural principles',
 'Front-end web application tech (e.g. javascript, CSS, html5, Vue, React/redux)',
 'Distributed cloud systems tech (e.g. node.js, EC2, Kubernetes, Lambda, Bedrock, DynamoDB, Elasticsearch)',
 'AI, Machine Learning and related data solutions',
 'Microservices and applications']

In [37]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [38]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mike, a business development executive at QuantumGen. QuantumGen is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of QuantumGen 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase QuantumGen's portfolio: {link_list}
        Remember you are Mike, BDE at QuantumGen. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Nike's Merchandising Functions

Dear Hiring Manager,

I came across the job posting for a Director of Software Engineering at Nike, and I believe QuantumGen's expertise in AI and software consulting can help fulfill your needs. With 14+ years of experience in leading high-performing software engineering teams, our team at QuantumGen can provide tailored solutions to enhance BU's merchandising functions.

Our team has extensive experience in cloud native technologies, Agile software development methodology, DevOps (CI/CD), and modern software architectural principles. We have successfully developed and implemented front-end web applications using technologies like JavaScript, CSS, HTML5, Vue, and React/Redux. Additionally, we have expertise in distributed cloud systems technologies such as Node.js, EC2, Kubernetes, Lambda, Bedrock, DynamoDB, and Elasticsearch.

Our AI and machine learning capabilities can help Nike leverage data solutio